In [ ]:
import pandas as pd
from google.cloud import bigquery
import time
import xml.etree.ElementTree as ET    #ET : xml 전처리해주는 도구
import requests
from google.oauth2 import service_account
import pandas_gbq

In [ ]:
!apt-get install google-cloud-sdk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package google-cloud-sdk


In [ ]:
start = time.time()

In [ ]:
#빅쿼리 연결 설정
project_id = 'strange-terra-415002'
client = bigquery.Client(project=project_id)

print('1/3 빅쿼리 연결 설정 완료')

1/3 빅쿼리 연결 설정 완료


In [ ]:
url = 'http://openapi.seoul.go.kr:8088/4178724f416e656437366d46676961/xml/tbLnOpendataRentV/1/5/2024'

# 파라미터를 딕셔너리 형태로 입력 : 강의 때 들은 방식(소괄호 사용)으로는 에러나서 불가능 했음(ValueError: too many values to unpack (expected 2))
params = {
    'KEY': '4178724f416e656437366d46676961',
    'TYPE': 'xml',
    'SERVICE': 'tbLnOpendataRentV',
    'START_INDEX': 0,
    'END_INDEX': 50
}

response = requests.get(url, params)

In [ ]:
#fromstring 메소드를 통해 XML 자료형을 처리할 수 있음
ET.fromstring(response.text)
print('2/3 데이터 생성 완료')

<Element 'tbLnOpendataRentV' at 0x7a3dd6e944f0>

In [ ]:
# XML을 DataFrame으로 변환
df = pd.read_xml(response.text)

# 필요한 컬럼 선택
columns_to_keep = ['ACC_YEAR', 'SGG_CD', 'SGG_NM', 'BJDONG_CD', 'BJDONG_NM', 'LAND_GBN', 'LAND_GBN_NM', 'BOBN', 'BUBN', 'FLR_NO', 'CNTRCT_DE', 'RENT_GBN', 'RENT_AREA', 'RENT_GTN', 'RENT_FEE', 'BLDG_NM', 'BUILD_YEAR', 'HOUSE_GBN_NM', 'CNTRCT_PRD', 'NEW_RON_SECD', 'CNTRCT_UPDT_RQEST_AT', 'BEFORE_GRNTY_AMOUNT', 'BEFORE_MT_RENT_CHRGE']
df = df[columns_to_keep]

# 결과 확인
usefull = ['ACC_YEAR', 'SGG_CD', 'SGG_NM', 'BJDONG_NM', 'FLR_NO', 'CNTRCT_DE', 'RENT_GBN', 'RENT_AREA', 'RENT_FEE', 'BLDG_NM','BUILD_YEAR', 'HOUSE_GBN_NM']
real_estate = df[usefull]

In [ ]:
table_id = 'strange-terra-415002.real_estate_auto.real_estate_table'          #corp_name따라 테이블 만들기
pandas_gbq.to_gbq(real_estate, table_id, project_id='strange-terra-415002', if_exists='replace') #project에 추가

100%|██████████| 1/1 [00:00<00:00, 7384.34it/s]


In [ ]:

end = time.time()
print("3/3 작업완료, {}초 소요".format(round(end-start)))

3/3 작업완료, 3287초 소요
